# Step by step processing of PIScO images from fullframe PNGs to EcoTaxa Export per cruise

## 0.  Connect to docker container & import of libraries
Prerequesits
- Start the rocm_container if not already running.
- Start the jupyter server in the container (sudo docker exec -it rocm_container bash -c "jupyter notebook --ip=0.0.0.0 --port=8888 --no-browser --allow-root"): note the token/password
- Find the ip address of the rocm container (sudo docker inspect -f '{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' rocm_container)
- Connect to the python kernel in VS Code: Select other kernel... with IP addres and password/token from above.

In [1]:
!pip install numpy pandas matplotlib scikit-image opencv-python umap-learn sqlalchemy tqdm standard-imghdr torch torchvision
import os
#os.chdir('/app/ImageProcessing/')
os.chdir('/home/veit/PIScO_dev/ImageProcessing/')

In [1]:
#### External modules
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import re
import datetime

from pandas.errors import EmptyDataError

from sqlalchemy import create_engine
from sqlalchemy import text

import inspect
from skimage import measure
from skimage.io import imread
import cv2
import umap
import pickle
from sklearn.preprocessing import StandardScaler

from tqdm import tqdm
import logging
import shutil

#### Internal modules

from segmenter import run_segmenter
import analyze_profiles_seavision as ap



/home/veit/miniconda3/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-10 11:14:20,641 - Connected to the database


## 1.  Define the input images i.e. cruise folder


In [2]:
#Don't forget to mount the SMB share, in system terminal: sudo mount -t cifs //filer.geomar.de/projekte/ORTC-ST-PISCO /mnt/filer -o username=GEOM-svcPISCO_rw@geomar.de,password=2mZmhx-7GCGf 
# Select the base dir of the PIScO data for the cruise you want to process. Example: /mnt/filer/SO298/SO298-Logfiles_PISCO

### HE570
cruise = "HE570"
cruise_base = "/mnt/filer/HE570/HE570-PISCO-Profiles"
intermediate_result_dir = f"/home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults"
ctd_dir = None
#ctd_prefix = "met_202_1_"
log_directory = None

### SO_298
# cruise = "SO298"
# cruise_base = "/mnt/filer/SO298/SO298-PISCO-Profiles"
# # #cruise_base = "/home/veit/PIScO_dev/usb_mount/SO298-PNG"
# intermediate_result_dir = "/home/veit/PIScO_dev/Segmentation_results/SO298/SO298-PISCO-Profiles/TempResults"
# ctd_dir = "/mnt/filer/SO298/SO298-CTD_UVP_ETC/SO298-CTD/calibrated/"
# ctd_prefix = "son_298_1_"
# log_directory = "/mnt/filer/SO298/SO298-Logfiles_PISCO/Templog"

### MSM_126
# cruise = "MSM126"
# cruise_base = "/mnt/filer/MSM126/MSM126-PISCO-Profiles"
# intermediate_result_dir = f"/home/veit/PIScO_dev/Segmentation_results/MSM126/MSM126-PISCO-Profiles/TempResults/"
# ctd_dir = "/mnt/filer/MSM126/MSM126-Data-UVP-CTD-ADCP/CTD/msm_126_1_ctd"
# ctd_prefix = "msm_126_1_"
# log_directory = "/mnt/filer/MSM126/MSM126-PISCO_Logfilesetc/Logfiles"

### M202
# cruise = "M202"
# cruise_base = "/mnt/filer/M202/M202-PISCO-Profiles"
# intermediate_result_dir = f"/home/veit/PIScO_dev/Segmentation_results/M202/M202-PISCO-Profiles/TempResults"
# ctd_dir = "/mnt/filer/M202/M202-external-Data/M202-CTD/met_202_1_ctd/met_202_1_ctd"
# ctd_prefix = "met_202_1_"
# log_directory = "/mnt/filer/M202/M202-Pisco-Logfiles/Logfiles"




## 2.  Main loop
### 2.1 Segmentation: create crops and corresponding unrefined metadata 


In [ ]:
for profile in os.listdir(cruise_base):
        print(profile)
        profile_path = os.path.join(cruise_base, profile)
        if os.path.isdir(profile_path):
                img_folder = os.path.join(profile_path, profile+"_Images-PNG")
                #img_folder = profile_path
                results_folder = os.path.join(intermediate_result_dir, profile, profile+"_Results")
                if not os.path.exists(img_folder):
                        print(f"Image folder {img_folder} does not exist, skipping profile {profile}.")
                        continue
                if not os.path.exists(results_folder):
                        os.makedirs(results_folder)
                run_segmenter(img_folder, results_folder, deconvolution=False)

### 2.2 Postprocessing: filtering, metadata curation, EcoTaxa export

In [5]:
def clean_ecotaxa_folders(base_folder):
        """
        Remove all EcoTaxa folders and their contents from the given base folder.
        
        Args:
            base_folder (str): Path to the base directory to clean
        """
        for root, dirs, _ in os.walk(base_folder):
            if "EcoTaxa" in dirs:
                ecotaxa_path = os.path.join(root, "EcoTaxa")
                print(f"Removing: {ecotaxa_path}")
                shutil.rmtree(ecotaxa_path)

clean_ecotaxa_folders(intermediate_result_dir)

In [ ]:
for profile in os.listdir(cruise_base):
    print(profile)
    profile_path = os.path.join(cruise_base, profile)
    profile_id = profile.split('_')[1]  # Extract the profile ID from the profile name
    if os.path.isdir(profile_path):
        # Some variables
        results_folder = os.path.join(intermediate_result_dir, profile, profile+"_Results")
        
        # if "crops_excluded" in os.listdir(results_folder):
        #     print(f"Profile {profile} already processed. Skipping.")
        #     continue
        profile_id = profile.split('_')[1]  # Extract the profile ID from the profile name
        profile_id = profile.split('_')[1]+profile.split('_')[2]  # HE570 profile ID is the second and the third part of the profile name
        profile_data_dir = os.path.join(results_folder, 'Data')

        # Load the results of the segmenter
        # and create a DataFrame with the particle data
        df = ap.gen_crop_df(profile_data_dir, small=False, size_filter=0, cruise=cruise)
        print(len(df.index), 'particles found.')
        df['fullframe_path'] = df['full_path'].apply(ap.modify_full_path)

        # Add ctd data to the DataFrame
        # Get CTD profile ID
        if ctd_dir is not None and os.path.exists(ctd_dir):
            ctd_id = ap.get_ctd_profile_id(cruise_base, profile)
            if ctd_id:
                ctd_file = os.path.join(ctd_dir, f'{ctd_prefix}{ctd_id}.ctd')
                print('adding ctd data...')
                df = ap.add_ctd_data(ctd_file, df)
            else:
                print(f"Skipping CTD data for {profile}")
                continue

        # Predict on deconvolved images                
        prediction_file = os.path.join(results_folder, 'ViT_predictions.csv')  # Example path
        if os.path.exists(prediction_file):
            print('Adding predictions...')
            prediction_df = pd.read_csv(prediction_file)
            df = ap.add_prediction(df, prediction_df)

        # Add log data to the DataFrame
        if log_directory is not None:                
            print('adding log info...')
            timestamp = profile.split('_')[-1]
            # Convert timestamp to datetime object
            date_time_obj = datetime.datetime.strptime(timestamp, '%Y%m%d-%H%M')
            min_diff = datetime.timedelta(days=365*1000)  # initialize with a big time difference
            closest_file = None

            # Iterate over all files in the directory
            for filename in os.listdir(log_directory):
                # Check if filename is a Templog
                if '__Templog.txt' in filename:
                    # Extract timestamp from filename and convert to datetime object
                    file_timestamp = filename[:16]
                    file_datetime = datetime.datetime.strptime(file_timestamp, '%Y%m%d_%Hh_%Mm')

                    # Calculate time difference
                    diff = abs(date_time_obj - file_datetime)

                    # If this file is closer, update min_diff and closest_file
                    if diff < min_diff:
                        min_diff = diff
                        closest_file = filename

            if closest_file is None:
                print("Logfile not found")
            else:
                file_path = os.path.join(log_directory, closest_file)
                file_size = os.path.getsize(file_path)  # Get file size in bytes
                print(f"Closest logfile: {closest_file}, Size: {file_size} bytes")
            
            # Read the log file and parse the relevant data

            df_log = ap.create_log_df(file_path)

            # Match the data with the profile dataframe
            df.drop(['TT_x', 'T1_x', 'T2_x', 'TH_x', 'restart_x', 'relock_x', 'Time_log_x', 'TT_y', 'T1_y', 'T2_y', 'TH_y', 'restart_y', 'relock_y', 'Time_log_y', 'TT', 'T1', 'T2', 'TH', 'restart', 'relock', 'Time_log'], axis=1, inplace=True, errors='ignore')
            # Convert the timestamps in both dataframes to datetime format
            df['timestamp'] = df['date-time']

            df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y%m%d-%H%M%S%f')

            # Sort the dataframes by the timestamp
            df = df.sort_values('timestamp')
            df_log = df_log.sort_values('timestamp')

            # Use merge_asof to merge the two dataframes, finding the nearest match on the timestamp
            df_combined = pd.merge_asof(df, df_log, left_on='timestamp', right_on='timestamp', direction='backward')
            df_combined.drop('timestamp', axis=1, inplace=True)

            #Add particle count based filter for filtering out images that are potentially obscured by schlieren or bubbles
            df_unique = df_combined[['date-time', 'pressure [dbar]', 'depth [m]', 'img_id','temperature','interpolated_s','interpolated_t','interpolated_o','interpolated_z_factor','restart','relock','TAG_event']].drop_duplicates()
            df_count = df_combined.groupby('date-time').size().reset_index(name='count')
            df_unique = df_unique.merge(df_count, on='date-time', how='left')
            df_unique = df_unique.sort_values('pressure [dbar]')
            df_unique['part_based_filter'] = df_unique['count'].apply(lambda x: 0 if x < (df_unique['count'].mean() + 5*df_unique['count'].std()) else 1)
            df = df_combined.merge(df_unique[['date-time', 'part_based_filter']], on='date-time', how='left')
        
        else:
            print('No log directory provided, skipping log data.')
            # Use merge_asof to merge the two dataframes, finding the nearest match on the timestamp
            df_combined = df.copy()

            #Add particle count based filter for filtering out images that are potentially obscured by schlieren or bubbles
            df_unique = df_combined[['date-time', 'pressure [dbar]', 'img_id']].drop_duplicates()
            df_count = df_combined.groupby('date-time').size().reset_index(name='count')
            df_unique = df_unique.merge(df_count, on='date-time', how='left')
            df_unique = df_unique.sort_values('pressure [dbar]')
            df_unique['part_based_filter'] = df_unique['count'].apply(lambda x: 0 if x < (df_unique['count'].mean() + 5*df_unique['count'].std()) else 1)
            df = df_combined.merge(df_unique[['date-time', 'part_based_filter']], on='date-time', how='left')


        ## Process for exporting to EcoTaxa
        # Add object ID column
        df['object_id'] = profile_id + df['img_id'].astype(str) + '_' + df['index'].astype(str) 

        #split date-time
        df[['date', 'time']] = df['date-time'].str.split('-', expand=True)

        # Get latitude and longitude from yaml file
        # Assuming your YAML file is in the same directory as your profile
        if cruise == "HE570":
            yaml_file = os.path.join(profile_path, profile + ".yaml")
        
        else:
            # For other cruises, the YAML file is in a subdirectory
            yaml_file = os.path.join(profile_path, profile + "_Metadata", profile + ".yaml")
        
        latitude, longitude = ap.extract_coords_from_yaml(yaml_file)

        # Add the columns to your DataFrame and fill all rows with the same values
        df['lat'] = latitude
        df['lon'] = longitude

        ## Filter crops based on aspect ratio and artifacts (TAG events and schlieren)
        threshold_ratio = 0.005  # Define the aspect ratio threshold
        print(len(df)-len(df[df['w'] >= threshold_ratio * df['h']]), 'crops removed due to aspect ratio')
        df = df[df['w'] >= threshold_ratio * df['h']]
        if "TAG_event" in df.columns:
            print(len(df)-len(df[df['TAG_event'] == 0]), 'crops removed due to TAG events')
            df = df[df['TAG_event'] == 0]

        # Filter out crops with part_based_filter == 1   
        df = df[df['part_based_filter'] == 0]  

        # Remove duplicates in the 'full_path' column, keeping the first occurrence
        df = df.drop_duplicates(subset='full_path', keep='first')
        df = df.sort_values(by='object_id')
        df.reset_index(drop=True, inplace=True)
        
        crops_zip = os.path.join(results_folder, "crops.zip")
        deconv_zip = os.path.join(results_folder, "deconv_crops.zip")
        
        # # Check if both zip files already exist
        # if os.path.exists(crops_zip) and os.path.exists(deconv_zip):
        #     print(f"Profile {profile_name} already processed. Skipping.")
        #     continue

        # Prepare EcoTaxa metadata
        df_ET = ap.rename_for_ecotaxa(df, sample_profile_id=profile).copy()
        if not df_ET.empty:
            ap.create_ecotaxa_zips(output_folder=results_folder, df=df_ET, profile_name=profile, 
                                max_zip_size_mb=500, compression_ratio=1.0, copy_images=True, 
                                add_scale_bar_to_deconv=False, pixel_resolution=23, scale_length_mm=1)
        else:
            print(f"No data to process for profile {profile}")

        #ap.create_ecotaxa_zips(output_folder=results_folder, df=df_ET, profile_name=profile, max_zip_size_mb=500, compression_ratio=1.0, copy_images=True, add_scale_bar_to_deconv=False, pixel_resolution=23, scale_length_mm=1)

        # Save the DataFrame to a CSV file
        df.to_csv(os.path.join(results_folder, f'{profile}_crops_metadata.csv'), index=False)

        # Create plots
        # plot_path = os.path.join(results_folder, 'Plots')
        # if not os.path.exists(plot_path):
        #     os.makedirs(plot_path)
        # ap.plot_histogram(df, plot_path)
        # ap.plot_position_hist(df, plot_path)
        # ap.plot_2d_histogram(df, plot_path)
        # press_min = df['object_pressure'].min()-10
        # depth_bin_size = 1
        # _, pivoted_df = ap.populate_esd_bins_pressure(df,  depth_bin_size=depth_bin_size, esd_bins=np.array([0,125,250,500,1000,100000]))
        # ap.plot_particle_dist(pivoted_df, folder, plot_path, depth_bin_size=depth_bin_size, preliminary=True, depth_min=press_min)
        # ap.plot_particle_dist(pivoted_df, folder, plot_path, depth_bin_size=depth_bin_size, preliminary=True, depth_min=press_min, maximum_y_value=500)
        # ap.plot_ctd_data(df, profile, plot_path)
        # ap.plot_ctd_data(df, profile, plot_path, maximum_y_value=500)

20210309_HE570_100-bop-019_PISCO1


  0%|          | 0/14257 [00:00<?, ?it/s]

100%|██████████| 14257/14257 [00:13<00:00, 1092.37it/s]


9846


2025-06-10 11:14:41,984 - crop_df created


...

0 files were empty and were dropped; Number of uniue images: 2388
33584 particles found.
No log directory provided, skipping log data.
0 crops removed due to aspect ratio
Processing 1 groups for /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210304_HE570_021-bop-003_PISCO1/20210304_HE570_021-bop-003_PISCO1_Results/Crops
Created zip file: /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210304_HE570_021-bop-003_PISCO1/20210304_HE570_021-bop-003_PISCO1_Results/EcoTaxa/crops.zip
Processing 1 groups for /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210304_HE570_021-bop-003_PISCO1/20210304_HE570_021-bop-003_PISCO1_Results/Deconv_crops
Created zip file: /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210304_HE570_021-bop-003_PISCO1/20210304_HE570_021-bop-003_PISCO1_Results/EcoTaxa/deconv_crops.zip
20210315_HE570_208-bop-036b_PISCO1


100%|██████████| 1697/1697 [00:02<00:00, 714.09it/s]
2025-06-10 11:29:30,482 - crop_df created


1575
0 files were empty and were dropped; Number of uniue images: 1575
11731 particles found.
No log directory provided, skipping log data.
0 crops removed due to aspect ratio
Processing 1 groups for /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210315_HE570_208-bop-036b_PISCO1/20210315_HE570_208-bop-036b_PISCO1_Results/Crops
Created zip file: /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210315_HE570_208-bop-036b_PISCO1/20210315_HE570_208-bop-036b_PISCO1_Results/EcoTaxa/crops.zip
Processing 1 groups for /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210315_HE570_208-bop-036b_PISCO1/20210315_HE570_208-bop-036b_PISCO1_Results/Deconv_crops
Created zip file: /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210315_HE570_208-bop-036b_PISCO1/20210315_HE570_208-bop-036b_PISCO1_Results/EcoTaxa/deconv_crops.zip
20210315_HE570_221-bop-039b_PISCO1


100%|██████████| 512/512 [00:00<00:00, 554.06it/s]
2025-06-10 11:29:37,892 - crop_df created


379
0 files were empty and were dropped; Number of uniue images: 379
1051 particles found.
No log directory provided, skipping log data.
0 crops removed due to aspect ratio
Processing 1 groups for /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210315_HE570_221-bop-039b_PISCO1/20210315_HE570_221-bop-039b_PISCO1_Results/Crops
Created zip file: /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210315_HE570_221-bop-039b_PISCO1/20210315_HE570_221-bop-039b_PISCO1_Results/EcoTaxa/crops.zip
Processing 1 groups for /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210315_HE570_221-bop-039b_PISCO1/20210315_HE570_221-bop-039b_PISCO1_Results/Deconv_crops
Created zip file: /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210315_HE570_221-bop-039b_PISCO1/20210315_HE570_221-bop-039b_PISCO1_Results/EcoTaxa/deconv_crops.zip


### 3. Upload to EcoTaxa

Setup of pyecotaxa 05.06.2025: 
1. Install pyecotaxa dev_branch: git clone https://github.com/moi90/pyecotaxa.git > cd pyecotaxa > git checkout wip-aqqua > pip install .
Now pyecotaxa is useable in cli and python scripts.
2. In home directory edit the file ~/.pyecotaxa.json: {"api_token": "...", "ftp_user": "ftp_plankton", "ftp_passwd": "Pl@nkt0n4Ecotaxa"}
3. Now ftp transfer is useable.

In [ ]:
import logging
import os
from pyecotaxa.remote import Remote, Transport

def login_to_ecotaxa(username, password):
    try:
        remote = Remote()
        remote.login(username, password)
        print("Successfully logged into EcoTaxa.")
        return remote
    except Exception as e:
        logging.error(f"Error during login: {e}")
        return None

def ET_upload(remote, project_id, folder_path):
    try:
        remote.push([(folder_path, project_id)],transport=Transport.FTP)
        print(f"Successfully uploaded {folder_path} to project {project_id}.")
    except Exception as e:
        logging.error(f"Error during upload: {e}") 

def upload_ecotaxa_data(profile_path, remote, project_id, crops_type='raw'):
    print(f"Processing EcoTaxa folder: {profile_path}")                
    zip_files = [f for f in os.listdir(profile_path) if f.endswith('.zip')]
    deconv_zip_files = [f for f in zip_files if 'deconv_crops' in f]
    raw_zip_files = [f for f in zip_files if 'crops' in f and 'deconv_crops' not in f]
    if crops_type == 'raw':
        # Check if there are zip files with "part" in their names
        part_zip_files = [f for f in raw_zip_files if "part" in f]

        if part_zip_files:
            # If "part" zip files exist, upload only those
            for part_zip in part_zip_files:
                zip_path = os.path.join(profile_path, part_zip)
                ET_upload(remote, project_id, zip_path)
                print(f"Uploaded {part_zip} to project {project_id}.")
        else:
            # Otherwise, upload the default crops.zip and deconv_crops.zip        
            zip_path_raw = os.path.join(profile_path, "crops.zip")
            ET_upload(remote, project_id, zip_path_raw)
            print(f"Uploaded crops.zip to project {project_id}.")
    else:
        # Upload deconv_crops.zip
        if not deconv_zip_files:
            logging.warning(f"No deconv_crops.zip found in {profile_path}.")
            return
        part_zip_files = [f for f in deconv_zip_files if "part" in f]

        if part_zip_files:
            # If "part" zip files exist, upload only those
            for part_zip in part_zip_files:
                zip_path = os.path.join(profile_path, part_zip)
                ET_upload(remote, project_id, zip_path)
                print(f"Uploaded {part_zip} to project {project_id}.")
        else:
        # Otherwise, upload the default deconv_crops.zip
            zip_path_deconv = os.path.join(profile_path, "deconv_crops.zip")
            ET_upload(remote, project_id, zip_path_deconv)
            print(f"Uploaded deconv_crops.zip to project {project_id}.")

USERNAME = 'vdausmann@geomar.de'  # Replace with your EcoTaxa username
PASSWORD = 'tEgso4-kukjyr-hotqab'  # Replace with your EcoTaxa password
# Login to EcoTaxa
remote = login_to_ecotaxa(USERNAME, PASSWORD)

for profile in os.listdir(cruise_base):
    print(profile)
    profile_path = os.path.join(intermediate_result_dir, profile, profile+"_Results", "EcoTaxa")
    if not os.path.exists(profile_path):
        print(f"EcoTaxa folder does not exist for profile {profile}. Skipping.")
        continue

    if remote:
        upload_ecotaxa_data(profile_path, remote, project_id=17302, crops_type='raw')
    else:
        print("Failed to log into EcoTaxa. Exiting...")

2025-06-10 11:35:34,418 - Server OpenAPI version is 0.0.41
2025-06-10 11:35:35,551 - Pushing 1 files...
2025-06-10 11:35:35,552 - Pushing /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210309_HE570_100-bop-019_PISCO1/20210309_HE570_100-bop-019_PISCO1_Results/EcoTaxa/crops.zip to 17302...


Successfully logged into EcoTaxa.
20210309_HE570_100-bop-019_PISCO1
Processing EcoTaxa folder: /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210309_HE570_100-bop-019_PISCO1/20210309_HE570_100-bop-019_PISCO1_Results/EcoTaxa


2025-06-10 11:35:35,552 - Uploading /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210309_HE570_100-bop-019_PISCO1/20210309_HE570_100-bop-019_PISCO1_Results/EcoTaxa/crops.zip via FTP...
  0%|          | 0/1 [00:00<?, ?it/s]2025-06-10 11:35:35,552 - Computing hash for crops.zip...
2025-06-10 11:35:35,903 - Remote filename is FTP/Ecotaxa_Data_to_import/pyecotaxa/a9ea74133508a373b486c8001b429f6834263f05548bfbeb57055f8d97735e4b/crops.zip.
2025-06-10 11:35:36,303 - Job to import FTP/Ecotaxa_Data_to_import/pyecotaxa/a9ea74133508a373b486c8001b429f6834263f05548bfbeb57055f8d97735e4b/crops.zip already exists...
100%|██████████| 1/1 [00:00<00:00,  1.33it/s]
2025-06-10 11:35:36,305 - Pushing 1 files...
2025-06-10 11:35:36,306 - Pushing /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210307_HE570_069-bop-015b_PISCO1/20210307_HE570_069-bop-015b_PISCO1_Results/EcoTaxa/crops.zip to 17302...


Successfully uploaded /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210309_HE570_100-bop-019_PISCO1/20210309_HE570_100-bop-019_PISCO1_Results/EcoTaxa/crops.zip to project 17302.
Uploaded crops.zip to project 17302.
20210307_HE570_069-bop-015b_PISCO1
Processing EcoTaxa folder: /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210307_HE570_069-bop-015b_PISCO1/20210307_HE570_069-bop-015b_PISCO1_Results/EcoTaxa


  0%|          | 0/1 [00:00<?, ?it/s]2025-06-10 11:35:36,306 - Uploading /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210307_HE570_069-bop-015b_PISCO1/20210307_HE570_069-bop-015b_PISCO1_Results/EcoTaxa/crops.zip via FTP...
2025-06-10 11:35:36,306 - Computing hash for crops.zip...
Uploading crops.zip...: 100%|██████████| 55.8M/55.8M [00:15<00:00, 3.82MB/s]
2025-06-10 11:35:52,092 - Remote filename is FTP/Ecotaxa_Data_to_import/pyecotaxa/955fc736f8323d3f6483cea9768ef05e45574b2583b41e016a021437ced69710/crops.zip.
2025-06-10 11:35:52,496 - Starting project import...
Importing to 17302... (Processing files 4800/5125):  95%|█████████▌| 95/100 [00:30<00:01,  3.13%/s]
100%|██████████| 1/1 [00:46<00:00, 46.70s/it]
2025-06-10 11:36:23,012 - Pushing 1 files...
2025-06-10 11:36:23,012 - Pushing /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210312_HE570_138-bop-028_PISCO1/20210312_HE570_138-bop-028_PISCO1_Results/EcoTaxa/cro

...

  0%|          | 0/1 [00:00<?, ?it/s]2025-06-10 13:05:11,011 - Computing hash for crops_part2_upload.zip...
Uploading crops_part2_upload.zip...: 100%|██████████| 58.7M/58.7M [00:09<00:00, 6.47MB/s]
2025-06-10 13:05:20,988 - Remote filename is FTP/Ecotaxa_Data_to_import/pyecotaxa/4d2e088d7d316f95b756a838cfd42c3baa0e0de9609b46fe6334e444faa53224/crops_part2_upload.zip.
2025-06-10 13:05:21,498 - Starting project import...
Importing to 17302... (Processing files 4000/4111):  98%|█████████▊| 98/100 [01:00<00:01,  1.62%/s]
100%|██████████| 1/1 [01:11<00:00, 71.29s/it]
2025-06-10 13:06:22,298 - Pushing 1 files...
2025-06-10 13:06:22,298 - Pushing /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210308_HE570_089-bop-018_PISCO1/20210308_HE570_089-bop-018_PISCO1_Results/EcoTaxa/crops.zip to 17302...
2025-06-10 13:06:22,299 - Uploading /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210308_HE570_089-bop-018_PISCO1/20210308_HE570_

Successfully uploaded /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210303_HE570_003-bop-001_PISCO1/20210303_HE570_003-bop-001_PISCO1_Results/EcoTaxa/crops_part2_upload.zip to project 17302.
Uploaded crops_part2_upload.zip to project 17302.
20210308_HE570_089-bop-018_PISCO1
Processing EcoTaxa folder: /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210308_HE570_089-bop-018_PISCO1/20210308_HE570_089-bop-018_PISCO1_Results/EcoTaxa


  0%|          | 0/1 [00:00<?, ?it/s]2025-06-10 13:06:22,299 - Computing hash for crops.zip...
Uploading crops.zip...: 100%|██████████| 54.8M/54.8M [00:03<00:00, 16.1MB/s]
2025-06-10 13:06:26,311 - Remote filename is FTP/Ecotaxa_Data_to_import/pyecotaxa/e78a66fb9efd55b4da760caa4bd7dec6c81a71e99f5788e265da677891d0b87a/crops.zip.
2025-06-10 13:06:26,649 - Starting project import...
Importing to 17302... (Processing files 11900/11912): 100%|██████████| 100/100 [01:25<00:00,  1.16%/s]
100%|██████████| 1/1 [01:30<00:00, 90.58s/it]
2025-06-10 13:07:52,884 - Pushing 1 files...


Successfully uploaded /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210308_HE570_089-bop-018_PISCO1/20210308_HE570_089-bop-018_PISCO1_Results/EcoTaxa/crops.zip to project 17302.
Uploaded crops.zip to project 17302.
20210305_HE570_044-bop-010_PISCO1
Processing EcoTaxa folder: /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210305_HE570_044-bop-010_PISCO1/20210305_HE570_044-bop-010_PISCO1_Results/EcoTaxa


2025-06-10 13:07:52,884 - Pushing /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210305_HE570_044-bop-010_PISCO1/20210305_HE570_044-bop-010_PISCO1_Results/EcoTaxa/crops.zip to 17302...
  0%|          | 0/1 [00:00<?, ?it/s]2025-06-10 13:07:52,885 - Uploading /home/veit/PIScO_dev/Segmentation_results/HE570/HE570-PISCO-Profiles/TempResults/20210305_HE570_044-bop-010_PISCO1/20210305_HE570_044-bop-010_PISCO1_Results/EcoTaxa/crops.zip via FTP...
2025-06-10 13:07:52,885 - Computing hash for crops.zip...
Uploading crops.zip...: 100%|██████████| 28.4M/28.4M [00:03<00:00, 9.88MB/s]
2025-06-10 13:07:56,381 - Remote filename is FTP/Ecotaxa_Data_to_import/pyecotaxa/19ae177a5c7fc397f25caf7ead0f3cf6c2de0461c2bbc93a565a8d3695506f9e/crops.zip.
2025-06-10 13:07:56,882 - Starting project import...
